In [4]:
import pymongo
from tqdm import tqdm
import os
import markdown
import re
from nltk.stem import PorterStemmer
st=PorterStemmer()

/home/li/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict


In [5]:
client = pymongo.MongoClient(host="192.168.0.106", port=29197)
client["github"].authenticate("github", "git332", "github")
db = client["github"]

In [6]:
db1 = db.composer_require_map
db2 = db.composer_require_set_detail
db3 = db.composer_tag_stem_top100

In [50]:
"php".find("php")

0

In [7]:
tag_map = {}
i = 0
for d in db1.find():
    i += 1
    if db2.find_one({"name":d["require"]}):
        d2 = db2.find_one({"name":d["require"]})
        if "keywords" in d2:
            for k in d2["keywords"]:
                if k.find("php") == -1:
                    k = st.stem(k)
                    if k in tag_map:
                        tag_map[k] += d["count"]
                    else:
                        tag_map[k] = d["count"]
# for d in db2.find():
#     i += 1
#     if "keywords" in d:
#         for k in d["keywords"]:
#             if k in tag_map:
#                 tag_map[k] += 1
#             else:
#                 tag_map[k] = 1
                
# for d in db3.find():
#     i += 1
#     if "keywords" in d:
#         for k in d["keywords"]:
#             if k in tag_map:
#                 tag_map[k] += 1
#             else:
#                 tag_map[k] = 1

In [8]:
tag_map

{'test': 6075,
 'librari': 994,
 'bdd': 1438,
 'tdd': 1333,
 'mockeri': 751,
 'mock': 1578,
 'stub': 796,
 'test doubl': 753,
 'mock object': 739,
 'test-doubl': 739,
 'proxi': 172,
 'trusted proxi': 124,
 'load balanc': 124,
 'framework': 1394,
 'laravel': 2693,
 'repl': 190,
 'psysh': 191,
 'tinker': 167,
 'xunit': 2831,
 'testing-tool': 2754,
 'except': 199,
 'error': 444,
 'handl': 236,
 'whoop': 170,
 'throwabl': 164,
 'error-handl': 219,
 'ui': 173,
 'symfoni': 5776,
 'cli': 1039,
 'consol': 513,
 'command-lin': 77,
 'artisan': 95,
 'laravel-zero': 72,
 'beyondcod': 48,
 'laravel-dump-serv': 44,
 'faker': 383,
 'fixtur': 530,
 'data': 368,
 'http': 1285,
 'rest': 582,
 'link': 26,
 'psr': 563,
 'psr-13': 85,
 'http-link': 5,
 'contain': 138,
 'container-interop': 76,
 'psr-11': 329,
 'container-interfac': 59,
 'log': 1017,
 'psr-3': 529,
 'compat': 167,
 'portabl': 120,
 'polyfil': 167,
 'shim': 100,
 'compon': 3627,
 'symfony-compon': 3589,
 'acl': 51,
 'secur': 466,
 'interfac'

In [9]:
composer_tag_top100 = sorted(tag_map.items(), key=lambda d: d[1], reverse=True)[:100]

In [10]:
composer_tag_top100

[('test', 6075),
 ('symfoni', 5776),
 ('compon', 3627),
 ('symfony-compon', 3589),
 ('xunit', 2831),
 ('testing-tool', 2754),
 ('laravel', 2693),
 ('mock', 1578),
 ('bdd', 1438),
 ('framework', 1394),
 ('tdd', 1333),
 ('symfony-bundl', 1309),
 ('http', 1285),
 ('bundl', 1085),
 ('cli', 1039),
 ('log', 1017),
 ('librari', 994),
 ('databas', 976),
 ('static-analysi', 862),
 ('stub', 796),
 ('standard', 760),
 ('test doubl', 753),
 ('mockeri', 751),
 ('mock object', 739),
 ('test-doubl', 739),
 ('psr-7', 716),
 ('coding-standard', 627),
 ('orm', 598),
 ('rest', 582),
 ('client', 575),
 ('psr', 563),
 ('qa', 549),
 ('fixtur', 530),
 ('psr-3', 529),
 ('consol', 513),
 ('autom', 502),
 ('persist', 498),
 ('cach', 482),
 ('twig', 479),
 ('valid', 478),
 ('secur', 466),
 ('templat', 463),
 ('error', 444),
 ('curl', 443),
 ('compos', 440),
 ('guzzl', 431),
 ('http client', 425),
 ('api', 407),
 ('doctrin', 395),
 ('httpclient', 391),
 ('yaml', 391),
 ('yii2', 390),
 ('faker', 383),
 ('cli-comma

In [11]:
for t in composer_tag_top100:
    db3.insert_one({"name":t[0],"count":t[1]})